**Desarrollado por:** Javier Fernando Botía Valderrama

*Docente del Departamento de Ingeniería de Sistemas*

**Materia:** Análisis Avanzado de Datos

**Departamento:** Ingeniería Aeroespacial

**Facultad de Ingeniería - Universidad de Antioquia**

# Agrupamiento Difuso

El agrupamiento difuso es un modelo que permite asociar o relacionar cada dato o elemento de un conjunto de datos a los grupos o clusters que se desean agrupar. A diferencia del agrupamiento generado por el algoritmo K-Means, el agrupamiento difuso establece la relación de los datos con los clusters o grupos mediante una **matriz de grados de pertenencia, $\mathbf{U}$**, el cual cada elemento de dicha matriz establece el grado de pertenencia de un elemento o dato a un cluster. La matriz $\mathbf{U}$ tiene la siguiente configuración:

$$\mathbf{U} = \begin{bmatrix} \mu_{1,1} & \cdots & \mu_{1,c} & \cdots &\mu_{1,K} \\ \vdots & \ddots & \vdots & \cdots & \vdots \\ \mu_{n,1} & \cdots & \mu_{n,c} & \cdots &\mu_{n,K} \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ \mu_{N,1} & \cdots & \mu_{N,c} & \cdots &\mu_{N,K}\end{bmatrix}_{N \, x\, K}$$

donde $\mu_{n,c}$ es el grado de pertenencia de una muestra o fila de la base de datos con respecto a una cluster $c$. La matriz $\mathbf{U}$ tiene un tamaño de $N$ muestras o filas de la base de datos por $K$ clusters o grupos. Cada grado de pertenencia tiene un valor entre $0$ y $1$, lo cual si un grado de pertenencia es igual a $1$, correspondería al máximo valor del grado de pertenencia. 

Si se desea extraer el *vector de clases* para representar el agrupamiento de datos desde la matriz $\mathbf{U}$, se calcula el *argumento del máximo valor* para cada muestra o fila de la base de datos:

$$Indice_n = arg \max_\limits{n}
   \lbrace \mu_{n,1}, \cdots, \mu_{n,c}, \cdots, \mu_{n,K}\rbrace$$

donde $Indice_n$ es la etiqueta de la clase donde se encontro el máximo grado de pertenencia de la muestra o fila de la base de datos $n$.

**Ejemplo:**

$$\mathbf{U} = \begin{bmatrix} 0.3 & 0.3 & 0.4 \\ 0.1 & 0.5 & 0.4 \\ 0.9 & 0.04 & 0.01 \\ 0.7 & 0.2 & 0.1\end{bmatrix}$$

$$Indice_{n=1} = arg \max_\limits{n = 1}
   \lbrace 0.3, 0.3, 0.4\rbrace = 0.4 (c = 3) = 3$$
$$Indice_{n=2} = arg \max_\limits{n = 2}
   \lbrace 0.1, 0.5, 0.4\rbrace = 0.5 (c = 2) = 2$$
$$Indice_{n=3} = arg \max_\limits{n = 3}
   \lbrace 0.9, 0.04, 0.01\rbrace = 0.9 (c = 1) = 1$$
$$Indice_{n=4} = arg \max_\limits{n = 4}
   \lbrace 0.7, 0.2, 0.1\rbrace = 0.7 (c = 1) = 1$$

$$Indice = \lbrace 3, 2, 1, 1 \rbrace$$

Uno de los algoritmos de agrupamiento difuso más populares es el algoritmo **Fuzzy C-Means** (Artículo original del algoritmo FCM: http://web-ext.u-aizu.ac.jp/course/bmclass/documents/FCM%20-%20The%20Fuzzy%20c-Means%20Clustering%20Algorithm.pdf), el cual esta basado en la teoría de los conjuntos difusos y el clásico algoritmo K-Means. Veamos un ejemplo práctico del uso de este algoritmo.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import requests
import io

## Carga de datos

Para esta parte practica, utilizaremos una base de datos en bruto de la calidad del aire

In [ ]:
url = "https://raw.githubusercontent.com/javierfernandobotia/AnalisisAvanzadoDatos/main/AirQualityUCI.csv"
download = requests.get(url).content
columnas = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15']
data = pd.read_csv(io.StringIO(download.decode('utf-8')),sep=';',decimal = ',', names = columnas)
# Seleccion[amos desde la característica tau1 hasta g4, para realizar la tarea de aprendizaje no supervisado
display(data.head())

In [ ]:
data = data.drop(columns=['F14','F15']) # Removemos las dos últimas columnas
display(data.head())

In [ ]:
data.info()

In [ ]:
from sklearn.impute import MissingIndicator

Indicador = MissingIndicator(missing_values = np.nan) # Decimos que datos debe buscar como datos faltantes
Indicador.fit(data) # Aplicamos el método de búsqueda con nuestra base de datos
Datos_Indicador = Indicador.transform(data)
POS = np.where(Datos_Indicador == True) # Se busca aquellos datos que tiene un valor Booleanos igual a True
print("Porcentaje de Datos Faltantes (%): ", 100*(len(POS[0])/(Datos_Indicador.shape[0]*Datos_Indicador.shape[1])))

In [ ]:
from sklearn.impute import SimpleImputer # Método de imputación 

imp = SimpleImputer(missing_values = np.nan, strategy= 'mean')
imp.fit(data)
Datos_Imputacion_Media = imp.transform(data)
Datos_Imputacion_Media = pd.DataFrame(Datos_Imputacion_Media, 
                                      columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13'])
Datos_Imputacion_Media.info()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

MM = MinMaxScaler(feature_range=(0, 1))
data_norm = MM.fit_transform(Datos_Imputacion_Media)
data_norm = pd.DataFrame(data_norm, columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13'])
display(data_norm)

## Algoritmo Fuzzy C-Means (FCM)

El primer paso es crear una función que permita aplicar el algoritmo FCM. Para este apartado, vamos a explorar el pseudocódigo del algoritmo FCM y después su implementación en código.

**Entrada:** $\lbrace Datos = X, Cluster = K, Defusificacion = m, Maxima\, iteracion = maxiter, Error = e\rbrace$ 

**Salida:** $\lbrace Matriz\, Pertenencia = U, Centros = V, Funcion = J, Error = e \rbrace$

1) Generar una matriz de **centros aleatorios**, $V$.

$$ V_{inicial} = \begin{bmatrix} v_{1,1} & \cdots & v_{1,d} & \cdots & v_{1,D} \\
\vdots & \ddots & \vdots & \cdots & \vdots \\ 
v_{c,1} & \cdots & v_{c,d} & \cdots & v_{c,D} \\
\vdots & \cdots & \vdots & \ddots & \vdots \\
v_{K,1} & \cdots & v_{k,d} & \cdots & v_{k,D} \end{bmatrix}_{K x D}$$

2) Calcular la distancia euclidiana, $d$.

$$d_{n,c} = \vert\vert x_n - v_c \vert\vert^2_{A = I} = (x_n - v_c)^T A \cdot(x_n - v_c) = (x_n - v_c)^T\cdot(x_n - v_c)$$

siendo $x_n$ un vector de los datos $X$, $I$ la matriz identidad, **$n$** una muestra y **$c$** un cluster.

3) Calcular los grados de pertenencia, $\mu_{n,c}$, organizando los valores en una matriz de grados de pertenencia, $U$

$$\mu_{n,c} = \left \lbrack \sum_{c=1}^{K = 1} \left(\frac{d_{n,j}}{d_{n,c}} \right )^{\frac{2}{m-1}} \right \rbrack ^{-1}$$

Ejemplo:

$$d = \begin{bmatrix} 0.65124238 & 0.84657243 & 0.55286725 & 0.43498079 \\
0.71870343 & 0.76755062 & 0.37233378 & 0.67988686 \end{bmatrix}$$
$Con\,\, m = 2$

$$d^{- \left (\frac{2}{m-1} \right)} = \begin{bmatrix} 2.35784197 & 1.39531337 & 3.27158545 & 5.28518065 \\
       1.93597864 & 1.69740683 & 7.21331857 & 2.16334959 \end{bmatrix}$$

$$\sum_{c=1}^{K = 2} d^{- \left (\frac{2}{m-1} \right)} = \begin{bmatrix} 4.29382061 & 3.09272019 & 10.48490403 & 7.44853025 \end{bmatrix}$$

$$\frac{d^{- \left (\frac{2}{m-1} \right)}}{\sum_{c=1}^{K = 2} d^{- \left (\frac{2}{m-1} \right)}} = \frac{ \begin{bmatrix} 2.35784197 & 1.39531337 & 3.27158545 & 5.28518065 \\
       1.93597864 & 1.69740683 & 7.21331857 & 2.16334959 \end{bmatrix}} {\begin{bmatrix} 4.29382061 & 3.09272019 & 10.48490403 & 7.44853025 \\
       4.29382061 & 3.09272019 & 10.48490403 & 7.44853025 \end{bmatrix}}$$
       
$$U = \begin{bmatrix} 0.54912447 & 0.45116056 & 0.31202817 & 0.70956021 \\
       0.45087553 & 0.54883944 & 0.68797183 & 0.29043979 \end{bmatrix}$$
       
$$\sum_{c=1}^{K = 2} \mu_{n,c} = \begin{bmatrix} 1 & 1 & 1 & 1\end{bmatrix}$$

4) Actualizar la matriz de centros, $V$, usando la siguiente ecuación:

$$v_c = \frac{\sum_{n=1}^{N} \mu_{n,c}^m \cdot x_{n}}{\sum_{n=1}^{N} \mu_{n,c}^m} = \frac{(\mu^m)^T x}{\sum_{n=1}^{N} \mu_{n,c}^m}$$

5) Calcular el valor de pérdida del agrupamiento, $J$, usando la siguiente ecuación:

$$J = \sum_{n=1}^{N} \sum_{c=1}^{K} \mu_{n,c}^{m} \cdot d_{n,c}^2$$

6) Calcular el error de agrupamiento, $\vert\vert U^{(i)} - U^{(i-1)}\vert \vert < e$. Si no se cumple, repetir los pasos 2), 3), 4), y 5) hasta alcanzar un error por debajo de $e$. 

Resultado Final del algoritmo FCM: Matriz de grados de pertenencia, $U$ y Matriz de centros actualizado, $V_{final}$.

$$U = \begin{bmatrix} \mu_{1,1} & \cdots & \mu_{1,c} & \cdots & \mu_{1,K} \\
\vdots & \ddots & \vdots & \cdots & \vdots \\
\mu_{n,1} & \cdots & \mu_{n,c} & \cdots & \mu_{n,K} \\ 
\vdots & \cdots & \vdots & \ddots & \vdots \\
\mu_{N,1} & \cdots & \mu_{N,c} & \cdots & \mu_{N,K}\end{bmatrix}_{N x K}$$

$$ V_{final} = \begin{bmatrix} v_{1,1} & \cdots & v_{1,d} & \cdots & v_{1,D} \\
\vdots & \ddots & \vdots & \cdots & \vdots \\ 
v_{c,1} & \cdots & v_{c,d} & \cdots & v_{c,D} \\
\vdots & \cdots & \vdots & \ddots & \vdots \\
v_{K,1} & \cdots & v_{k,d} & \cdots & v_{k,D} \end{bmatrix}_{K x D}$$

Una vez se finalice el algoritmo FCM, se procede a calcular el vector de clases para visualizar el agrupamiento de los datos, por medio del cálculo del **máximo grado de pertenencia en cada muestra** $n$:

$$Clases_n = arg \max \lbrace \mu_{n,1},\ldots, \mu_{n,c}, \ldots, \mu_{n,K} \rbrace $$

**Ejemplo del cálculo del máximo grado de pertenencia en cada muestra**:

$$U = \begin{bmatrix} 0.3 & 0.2 & 0.5 \\
0.4 & 0.3 & 0.3 \\
0.3 & 0.5 & 0.2 \\
0.1 & 0.7 & 0.2 \\ \end{bmatrix}_{4 x 3}$$

$Clases_{n = 1} = arg \max \lbrace 0.3(c=1), 0.2(c=2), 0.5(c=3) \rbrace  = 0.5 (c=3) = 3$
$Clases_{n = 2} = arg \max \lbrace 0.4(c=1), 0.3(c=2), 0.3(c=3) \rbrace  = 0.4 (c=1) = 1$
$Clases_{n = 3} = arg \max \lbrace 0.3(c=1), 0.5(c=2), 0.2(c=3) \rbrace  = 0.5 (c=2) = 2$
$Clases_{n = 4} = arg \max \lbrace 0.1(c=1), 0.7(c=2), 0.2(c=3) \rbrace  = 0.7 (c=2) = 2$

Por consiguiente, el vector de clases es:

$Clases = \lbrace 3, 1, 2, 2\rbrace$

A partir del pseudocódigo, se crea una función llamada **FCM_Inicial**, para obtener una matriz de grados de pertenencia inicial, **u_0**, y el valor de pérdida de agrupamiento inicial **J**. Recuerden que la matriz de centros se genera aleatoriamente para iniciar el algoritmo.

In [ ]:
# X = datos, K = número de clusters, y m = parámetro de difusificación
def FCM_Inicial(X, K, m):
  import numpy as np
  from scipy.spatial import distance
  D = X.shape[1]
  centros = np.random.rand(K,D) # Generar una matriz de centros aleatorios.
  d = distance.cdist(X, centros) # Cálculo de la distancia Euclidiana entre los datos y los centros.
  d = d.T
  u_0 = d ** (- 2. / (m - 1)) # calculo de la distancia euclidiana elevado a la potencia (-2 / (m-1))
  u_0 /= np.ones((K, 1)).dot(np.atleast_2d(u_0.sum(axis=0))) # Matriz de grados de pertenencia inicial
  um = (u_0)**m
  J = (um*(d**2)).sum() # calculo del valor de pérdida del agrupamiento
  return u_0, J

Luego, se crea una función llamada **FCM_Update**, para actualizar la matriz de centros y la matriz de grados de pertenencia, así como el cálculo de la pérdida de agrupamiento actualizado.

In [ ]:
def FCM_Update(X, K, m, u):
    import numpy as np
    from scipy.spatial import distance
    um = u**m
    centros = um.dot(X) / (np.ones((X.shape[1],1)).dot(np.atleast_2d(um.sum(axis=1))).T) # Actualizar centros
    d = distance.cdist(X, centros) # Cálculo de la distancia Euclidiana entre los datos y los centros
    d = d.T
    u_actual = d ** (- 2. / (m - 1))
    u_actual /= np.ones((K, 1)).dot(np.atleast_2d(u_actual.sum(axis=0))) # Actualizar matriz de grados de pertenencia U
    um = (u_actual)**m
    J = (um*(d**2)).sum()
   
    return u_actual, centros, J

Finalmente, se crea el programa principal llamado **FCM_main**, donde se integra **FCM_Inicial** y **FCM_Update**.

In [ ]:
def FCM_main(X, K, m, error, max_iter):
    J = []
    err = []
    U_0, J_0 = FCM_Inicial(X, K, m) # Agrupamiento inicial
    J.append(J_0)
    i = 0
    for i in range(max_iter):
        if i == 0:
            u_actual, centros, J_1 = FCM_Update(X, K, m, U_0) # Actualización de la matriz U y la matriz V
            J.append(J_1) # Se almacena el valor de la pérdida del agrupamiento.
            u_1 = u_actual
            errores = np.linalg.norm(u_1 - U_0)  # Se calcula la norma entre U_0 o matriz de grados de pertenencia inicial
                                                 # y la matriz de grados de pertenencia actualizada u_1
            err.append(errores)
            if errores < error: # Se determina si se comple con la condición de finalización.
                break
        if i > 0:
            u_2, centros2, J_2 = FCM_Update(X, K, m, u_1) # Actualización de la matriz U y la matriz V
            J.append(J_2) # Se almacena el valor de la pérdida del agrupamiento.
            errores = np.linalg.norm(u_2 - u_1) # Se calcula la norma entre las matrices de grados de pertenencia u_2 y u_1
            err.append(errores)
            if errores < error: # Se determina si se comple con la condición de finalización.
                break
            u_1 = u_2
    return u_2.T, centros2, J, err # Se genera como resultado final la matriz de grados de pertenencia, la matriz de centros,
                                   # e

Una vez se construye el algoritmo FCM, se utiliza **FCM_main** para agrupar los datos, definiendo los siguientes parámetros para su uso:

In [ ]:
X = data_norm # Datos
K = 2 # Mumero de clusters
m = 2 # Indice de dedifusificación del algoritmo FCM
error = 1e-4 # Error mínimo del agrupamiento
max_iter = 1000 # Máximo Número de Iteraciones

U, center, J, err = FCM_main(X, K, m, error, max_iter) # Algoritmo FCM

En la siguiente grafica, se puede apreciar el comportamiento de la pérdida del agrupamiento, $J$, con respecto al número de iteraciones del algoritmo FCM.

In [ ]:
plt.plot(np.arange(len(J)), J, color="r") # Generar gráfica con color rojo
plt.axhline(0, color="blue") # Elegir color de la linea horizontal de referencia
plt.title('Perdida del agrupamiento - Cluster = {:.2f}'.format(K))
plt.xlabel('Iteraciones') # Etiqueta del eje x
plt.ylabel('Pérdida del Agrupamiento') # Etiqueta del eje y
plt.show() 

En esta grafica, se puede apreciar como el error del agrupamiento va decreciendo a medida que se incrementa el número de iteraciones. En algunos casos, se puede generar algunas oscilaciones debido a la pérdida del agrupamiento en cada iteración y la actualización de la matriz de centros del algoritmo FCM.

In [ ]:
plt.plot(np.arange(len(err)), err, color="m") # Generar gráfica con color rojo
plt.axhline(0, color="blue") # Elegir color de la linea horizontal de referencia
plt.axhline(error, color="r") # Elegir color de la linea horizontal de referencia del error dado por el usuario
plt.title('Error de Agrupamiento - Cluster = {:.2f}'.format(K))
plt.xlabel('Iteraciones') # Etiqueta del eje x
plt.ylabel('Error') # Etiqueta del eje y
plt.show()

A partir de la matriz de grados de pertenencia, $U$, se obtiene el vector de clases.

In [ ]:
clases = np.argmax(U, axis=1) # Genera el vector de clases a partir del argumento del máximo valor de una matriz
print("Vector de clases = ",clases) 

Seleccionamos dos características de la base de datos para graficar el espacio de características que permita visualizar el agrupamiento de datos generado por el algoritmo FCM:

In [ ]:
x_1 = data_norm['F11']
x_2 = data_norm['F12']
colors = ["b", "orange"]
fig = plt.figure(figsize = (5,5))

for j in range(K):
    plt.plot(x_1[clases == j], x_2[clases == j], '.', color = colors[j])

for pt in center:
    plt.plot(pt[0],pt[1],'rs')

plt.xlabel("F11")
plt.ylabel("F12")
plt.title("Agrupamiento de los datos")
plt.show()

Como se explico en el algoritmo K-Means, es necesario buscar el número óptimo de clusters. Sin embargo, como el agrupamiento difuso es diferente al algoritmo que genera el algoritmo K-Means debido a la matriz de grados de pertenencia, es necesario usar otros índices de validación interna para el algoritmo FCM.



## Indices de Validación Interna para Algoritmos de Agrupamiento Difuso

El **Coeficiente de Partición (PC)** es una métrica que mide el grado de difuso de una matriz de grados de pertenencias, $\mathbf{U}$. El máximo valor de $PC$ permite obtener el número óptimo de clusters. 


In [ ]:
def PC(u):
  import numpy as np
  N = u.shape[0]
  pc = (1/N)*(np.square(u).sum())
  return pc

El **Coeficiente de Entropía (PE)** es una métrica que mide el grado de difuso de una matriz de grados de pertenencias, $\mathbf{U}$, usando la métrica de entropia de Shannon. El mínimo valor de $PE$ permite obtener el número óptimo de clusters.

In [ ]:
def PE(u):
  import numpy as np
  N = u.shape[0]
  pe = -(1/N)*(u*np.log(u)).sum()
  return pe

El **Coeficiente de Partición Modificado (MPC)** es una mejora de la métrica PC que considera la dependencia de la matriz de grados de pertenencia con respecto al número de clusters. El máximo valor de $MPC$ permite obtener el número óptimo de clusters.

In [ ]:
def MPC(u):
  import numpy as np
  N = u.shape[0]
  C = u.shape[1]
  pc = (1/N)*(np.square(u).sum())
  mpc = 1 - ((C/(C - 1))*(1 - pc))
  return mpc

El **Coeficiente de Entropía Modificado (MPE)** es una mejora de la métrica $PE$ que considera la dependencia de la matriz de grados de pertenencia con respecto al número de clusters y el número de muestras de la base de datos. El mínimo valor de $MPE$ permite obtener el número óptimo de clusters.

In [ ]:
def MPE(u):
  import numpy as np
  N = u.shape[0]
  C = u.shape[1]
  pe = -(1/N)*(u*np.log(u)).sum()
  mpe = (N*pe)/(N-C)
  return mpe

El índice **Xie-Beni (XB)** es una métrica que considera la matriz de grados de pertenencia y la matriz de datos, lo cual permite obtener un valor promedio entre la compactación y la separabilidad de los clusters. El mínimo valor de $XB$ permite obtener el número óptimo de clusters.

In [ ]:
def XB(u,x,m,center):
  from scipy.spatial import distance
  import numpy as np
  N = u.shape[0]
  C = u.shape[1]
  u_m = u**m
  dist_x_v = distance.cdist(x, center)
  dist_v_v = distance.cdist(center, center)
  dist_v_v[dist_v_v == 0] = np.inf # Cuando la distancia de un centro de una clase 1 con el mismo centro
                                   # es decir, d(v1,v1) = 0, entonces se iguala a un valor inf que evita
                                   # errores en el cálculo de la mínima distancia entre centros de diferente
                                   # cluster o grupo
  xb = (u_m*dist_x_v).sum()/(N*dist_v_v.min())
  return xb

El índice **Fukuyama - Sugeno (FS)** es una métrica que permite evaluar la compactación y la separabilidad entre clusters, relacionando una medida de la pérdida del agrupamiento y una medida de la separabilidad entre clusters con respecto a la matriz de grados de pertenencia. El valor mínimo de $FS$ permite obtener el número óptimo de clusters.

In [ ]:
def FS(u,x,m,center):
  from scipy.spatial import distance
  import numpy as np
  N = u.shape[0]
  C = u.shape[1]
  u_m = u**m
  media_centros = center.mean(axis = 0)
  dist_x_v = distance.cdist(x, center)
  dist_square_v_vmedia = np.linalg.norm(center - media_centros, axis=1, keepdims=True)**2
  fs = (u_m*dist_x_v).sum() - (u_m.T*dist_square_v_vmedia).sum()
  return fs

El índice **simétrico entre clusters (SYM)** es una métrica que evalua la calidad del agrupamiento difuso a partir del calculo de los puntos simétricos de cada cluster y la máxima distancia entre los centros. El valor máximo de $SYM$ permite obtener el número óptimo de clusters.

In [ ]:
def SYM_INDEX(u,x,center):
  from scipy.spatial import distance
  import numpy as np
  C = u.shape[1]
  dist_x_v = distance.cdist(x, center)
  E_K = (u*dist_x_v).sum()
  dist_v_v = distance.cdist(center, center)
  dist_v_v[dist_v_v == 0] = 0
  D_K = dist_v_v.max()
  syms_index = (1/C)*(1/E_K)*D_K
  return syms_index

El índice **Wu - Li (WLI)** es una métrica que considera una medida de compactación por medio de una ponderación de las distancias difusas de los centros con los datos y la cardinalidad difusa de cada cluster (recuerden que en la teoría de conjuntos, la cardinalidad mide la cantidad de elementos de un conjunto). La separabilidad de esta métrica se establece mediante una distancia entre los centros, estableciendo un valor promedio entre la mínima distancia y la distancia mediana de los clusters. Al establecer una relación entre compactación y la separabilidad, el mínimo valor de $WLI$ permite establecer el número óptimo de clusters. 

In [ ]:
def WLI(u,x,center):
  import numpy as np
  from scipy.spatial import distance
  dist_x_v = distance.cdist(x, center)
  u_2 = u**2
  L = (u_2*dist_x_v**2).sum(axis = 0)/u.sum(axis = 0)
  WLn = L.sum()
  dist_v_v = distance.cdist(center, center)
  dvv = dist_v_v**2
  dvv[dvv == 0] = np.inf
  nuevo_dvv = np.delete(dvv,range(0,dvv.shape[0]**2,(dvv.shape[0]+1))).reshape(dvv.shape[0],(dvv.shape[1]-1))
  WLd = (dvv.min() + np.median(nuevo_dvv))/2
  wli = WLn/2*WLd
  return wli 

## Búsqueda del Número Óptimo de Clusters en el Modelo del Agrupamiento Difuso

In [ ]:
data_norm.shape

In [ ]:
display(data_norm.head(5))

In [ ]:
fig1,axes1 = plt.subplots(3,3,figsize = (8,8))

x_1 = data_norm['F11']
x_2 = data_norm['F12']
feature_A = 10 # Ubicación equivalente a 'F11'. Recuerde que en un arreglo, la posición inicia desde 0, por esto se define que
               # 'F11' esta ubicado en la columna 10 de la variable data_norm. Esto será útil para ubicar los centros.
feature_B = 11 # Ubicación equivalente a 'F12'

U = []
C = []
Perdida_Agrupamiento = []
Error_Agrupamiento = []
Vector_Clases = []

Tabla = []

colors = ["b", "orange", "g", "indigo", "c", "m", "y", "k", "Brown", "ForestGreen"]

m = 2
error = 1e-4
max_iter = 2000

for ncenters, ax in enumerate(axes1.reshape(-1), 2):
  u, centros, J, err = FCM_main(data_norm, ncenters, m = m, error = error, max_iter = max_iter)
  clases = np.argmax(u, axis=1) # Extraemos el vector de clases a partir de la matriz U
  Vector_Clases.append(clases)
  U.append(u)
  C.append(centros)
  Perdida_Agrupamiento.append(J)
  Error_Agrupamiento.append(err)
  pc = PC(u)
  pe = PE(u)
  mpc = MPC(u)
  mpe = MPE(u)
  xb = XB(u,data_norm,m,centros)
  fs = FS(u,data_norm,m,centros)
  syms_index = SYM_INDEX(u,data_norm,centros)
  wli = WLI(u,data_norm,centros)
  Metricas = [ncenters, pc, pe, mpc, mpe, xb, fs, syms_index, wli]
  Tabla.append(Metricas)

  for j in range(ncenters):
    ax.plot(x_1[clases == j], x_2[clases == j], '.', color = colors[j])
  
  for pt in centros:
    ax.plot(pt[feature_A],pt[feature_B],'rs')

  ax.set_title('Cluster = {0}'.format(ncenters))
  ax.axis('off')

fig1.tight_layout()
plt.show()

In [ ]:
for j in range(ncenters-1):
  plt.plot(np.arange(len(Perdida_Agrupamiento[j-2])), Perdida_Agrupamiento[j-2], color="r") # Generar gráfica con color rojo
  plt.axhline(0, color="blue") # Elegir color de la linea horizontal de referencia
  plt.title('Perdida del agrupamiento - Cluster = {:.2f}'.format(j+2))
  plt.xlabel('Iteraciones') # Etiqueta del eje x
  plt.ylabel('Pérdida del Agrupamiento') # Etiqueta del eje y
  plt.show()

In [ ]:
for j in range(ncenters-1):
  post = len(Error_Agrupamiento[j-2]) 
  Er = pd.DataFrame(Error_Agrupamiento[j-2])
  print("Error del Agrupamiento = ",Er[post-1:post].values)
  plt.plot(np.arange(len(Error_Agrupamiento[j-2])), Error_Agrupamiento[j-2], color="m") # Generar gráfica con color rojo
  plt.axhline(0, color="blue") # Elegir color de la linea horizontal de referencia
  plt.title('Error del agrupamiento - Cluster = {:.2f}'.format(j+2))
  plt.xlabel('Iteraciones') # Etiqueta del eje x
  plt.ylabel('Error del Agrupamiento') # Etiqueta del eje y
  plt.show()

In [ ]:
COL = ['Clusters', 'PC', 'PE', 'MPC', 'MPE', 'XB', 'FS', 'SYMS', 'WLI']
Tabla = pd.DataFrame(Tabla, columns = COL)
display(Tabla)

In [ ]:
Valores_Optimos = [Tabla['PC'].max(), Tabla['PE'].min(), Tabla['MPC'].max(), Tabla['MPE'].min(), Tabla['XB'].min(), Tabla['FS'].min(),
                   Tabla['SYMS'].max(), Tabla['WLI'].min()]
pos_Optimos = [np.argmax(Tabla['PC']), np.argmin(Tabla['PE']), np.argmax(Tabla['MPC']), np.argmin(Tabla['MPE']),
               np.argmin(Tabla['XB']), np.argmin(Tabla['FS']), np.argmax(Tabla['SYMS']), np.argmin(Tabla['WLI'])]
Clust = Tabla['Clusters']
Clusters_Optimos = Clust[pos_Optimos]
Clusters_Optimos = Clusters_Optimos.reset_index()
Clusters_Optimos = Clusters_Optimos['Clusters'].values
Valores_Optimos = np.array(Valores_Optimos)
Tabla_Resumen = pd.DataFrame(np.vstack((Valores_Optimos, Clusters_Optimos)),columns = ['PC', 'PE','MPC', 'MPE', 'XB', 'FS','SYMS','WLI'], 
                             index = ['Valor Métrica','Clusters óptimos'])
display(Tabla_Resumen)

In [ ]:
Etiquetas_Seleccionadas = Vector_Clases[0] # 0 es la posición donde esta ubicado el número óptimo de clusters

In [ ]:
Datos_Finales = pd.concat([Datos_Imputacion_Media,pd.DataFrame(Etiquetas_Seleccionadas,columns = ['Clase'])], axis = 1)
display(Datos_Finales)